# Fine-Structure Constant (α)

Solve cubic equation with A = 1/(256π³), κ = (41/20π)ln(1/φ₀)

**Formula:** $solveCubic(1, -A, 0, -A * c₃² κ)$

**Unit:** dimensionless

**Category:** fundamental

In [1]:
import sympy as sp
import numpy as np
from sympy import symbols, pi, sqrt, acos, asin, atan, exp, log, sin, cos
import pint
import json
from pathlib import Path
import os
import warnings
warnings.filterwarnings('ignore')  # Suppress numpy warnings for cleaner output

# Initialize unit registry
ureg = pint.UnitRegistry()
Q_ = ureg.Quantity

# Define custom units if needed
try:
    # Nuclear magneton (in SI units: J/T)
    ureg.define('nuclear_magneton = 5.050783699e-27 * joule / tesla')
    ureg.define('μ_N = nuclear_magneton')  # Greek mu with subscript N
except:
    pass  # Unit might already be defined

# Bootstrap commonly used constants that may not be in dependencies
# These are fundamental constants that many calculations rely on
BOOTSTRAP_CONSTANTS = {
    'DISPLAY_SCALE': 1e10,  # GeV display scale
    'RG_FACTOR': 1.0,       # Renormalization group factor
    'm_e_MEV': 0.51099895,  # Electron mass in MeV
    'hbar': 1.0,            # Natural units (ħ=c=1)
    'c': 1.0,               # Natural units (ħ=c=1)
    'G': 6.67430e-11,       # Gravitational constant (SI)
    'b_Y': 41.0/10.0,       # SM beta function coefficient
    'g_1': 0.3583,          # U(1) coupling at M_Z
    'g_2': 0.6536,          # SU(2) coupling at M_Z
    'Y': 1.0,               # Yukawa coupling (placeholder)
    'T_PLANCK': 1.416784e32, # Planck temperature in K
    'REHEAT_FACTOR': 1e-16,  # Typical reheating factor
    'BETA_0': 41.0/10.0,     # Same as b_Y
    'H_0': 2.2e-18,          # Hubble constant in SI units (Hz)
    'LEPTONIC': 0.17,        # Leptonic branching ratio for tau
    'HADRONIC': 0.65,       # Hadronic branching ratio for tau
    'E8_FACTOR': 5.0/6.0,    # E8 factor for tau mass
    'hbar_GeVs': 6.582119e-25,  # Reduced Planck constant in GeV*s
    'hbar': 1.0,             # Natural units (ħ=c=1)
    'c': 299792458.0,        # Speed of light in m/s
}

# Load constant metadata
# Handle different working directories (local vs Docker)
possible_paths = [
    Path('../constants/data/alpha.json'),
    Path('/app/constants/data/alpha.json'),
    Path('constants/data/alpha.json'),
    Path('./data/alpha.json')
]

metadata = None
for const_path in possible_paths:
    if const_path.exists():
        with open(const_path, 'r') as f:
            metadata = json.load(f)
        break

if metadata is None:
    raise FileNotFoundError(f"Could not find alpha.json in any expected location")


In [2]:
# Step 1: Define symbols
alpha, c_3 = symbols('alpha, c_3', real=True, positive=True)

In [3]:
# Step 2: Define cubic equation for alpha
# α³ - Aα² - Ac₃²κ = 0
A, kappa = symbols('A kappa', real=True, positive=True)
cubic_eq = alpha**3 - A*alpha**2 - A*c_3**2*kappa
print(f'Cubic equation: {cubic_eq} = 0')
print(f'LaTeX: {sp.latex(cubic_eq)} = 0')

Cubic equation: -A*alpha**2 - A*c_3**2*kappa + alpha**3 = 0
LaTeX: - A \alpha^{2} - A c_{3}^{2} \kappa + \alpha^{3} = 0


In [4]:
# Step 3: Load dependency values
dependency_values = {}

# Load Fundamental VEV
# Try multiple possible paths for phi_0.json
dep_paths = [
    Path('../constants/data/phi_0.json'),
    Path('/app/constants/data/phi_0.json'),
    Path('constants/data/phi_0.json'),
    Path('./data/phi_0.json')
]
for dep_path in dep_paths:
    if dep_path.exists():
        with open(dep_path, 'r') as f:
            phi_0_data = json.load(f)
        break
else:
    raise FileNotFoundError(f'Could not find phi_0.json')
dependency_values['phi_0'] = phi_0_data['sources'][0]['value']
print(f"phi_0 = {dependency_values['phi_0']}")

# Load Topological Fixed Point
# Try multiple possible paths for c_3.json
dep_paths = [
    Path('../constants/data/c_3.json'),
    Path('/app/constants/data/c_3.json'),
    Path('constants/data/c_3.json'),
    Path('./data/c_3.json')
]
for dep_path in dep_paths:
    if dep_path.exists():
        with open(dep_path, 'r') as f:
            c_3_data = json.load(f)
        break
else:
    raise FileNotFoundError(f'Could not find c_3.json')
dependency_values['c_3'] = c_3_data['sources'][0]['value']
print(f"c_3 = {dependency_values['c_3']}")

phi_0 = 0.053171
c_3 = 0.039788735772973836


In [5]:
# Step 4: Calculate alpha by solving cubic equation
# Define A and κ from theory
A = 1 / (256 * pi**3)
kappa = (41 / (20 * pi)) * log(1 / dependency_values['phi_0'])

print(f'A = {float(A)}')
print(f'κ = {float(kappa)}')

# Substitute into cubic equation
# Find c_3 symbol
c_3_value = dependency_values.get('c_3', None)
if c_3_value is None:
    raise ValueError("c_3 dependency value not found")

# Create substitution dictionary with numeric values
substitutions = {
    'A': float(A),
    'kappa': float(kappa),
    'c_3': c_3_value
}

# Apply substitutions
numeric_eq = cubic_eq
for sym in cubic_eq.free_symbols:
    if str(sym) in substitutions:
        numeric_eq = numeric_eq.subs(sym, substitutions[str(sym)])

print(f'Equation to solve: {numeric_eq} = 0')

# Solve cubic equation α³ - Aα² - Ac₃²κ = 0 using numpy polynomial solver
import numpy as np

# Create polynomial coefficients [highest degree first]
# α³ - Aα² - Ac₃²κ = 0 => coeffs = [1, -A, 0, -A*c₃²*κ]
A_val = float(A)
c3_val = c_3_value
kappa_val = float(kappa)

coeffs = [1.0, -A_val, 0.0, -A_val * c3_val**2 * kappa_val]
print(f'\nPolynomial coefficients: {coeffs}')

# Find all roots
all_roots = np.roots(coeffs)
print(f'\nAll roots from polynomial solver:')
for i, root in enumerate(all_roots):
    if np.isreal(root):
        print(f'  Root {i+1}: {float(np.real(root))}')
    else:
        print(f'  Root {i+1}: {root} (complex)')

# Filter for positive real roots
real_positive_roots = []
for root in all_roots:
    if np.isreal(root) and np.real(root) > 0:
        real_positive_roots.append(float(np.real(root)))

real_positive_roots.sort()
print(f'\nPositive real roots: {real_positive_roots}')

# Selection criterion: largest perturbative root (α < 0.01)
perturbative_roots = [r for r in real_positive_roots if r < 0.01]
if perturbative_roots:
    calculated_value = max(perturbative_roots)
    print(f'\nSelected root (largest perturbative): α = {calculated_value}')
else:
    # If no perturbative roots, take the smallest positive root
    if real_positive_roots:
        calculated_value = min(real_positive_roots)
        print(f'\nNo perturbative roots found, using smallest positive: α = {calculated_value}')
    else:
        raise ValueError('No positive real solutions found for alpha')

# Verification: check that the root satisfies the equation
residual = calculated_value**3 - A_val * calculated_value**2 - A_val * c3_val**2 * kappa_val
print(f'\nVerification - cubic residual: {residual}')
print(f'Relative error: {abs(residual/calculated_value):.2e}')

A = 0.0001259825563796855
κ = 1.9146964799582853
Equation to solve: alpha**3 - 0.000125982556379686*alpha**2 - 3.81883272991332e-7 = 0

Polynomial coefficients: [1.0, -0.0001259825563796855, 0.0, -3.818832729913318e-07]

All roots from polynomial solver:
  Root 1: 0.007297340575733252
  Root 2: (-0.003585679009676779+0.0062828932743841884j) (complex)
  Root 3: (-0.003585679009676779-0.0062828932743841884j) (complex)

Positive real roots: [0.007297340575733252]

Selected root (largest perturbative): α = 0.007297340575733252

Verification - cubic residual: 1.5352472168984194e-21
Relative error: 2.10e-19


In [6]:
# Step 5: Compare with reference value
reference_value = metadata['sources'][0]['value']
relative_error = abs(calculated_value - reference_value) / reference_value

print(f'Reference value: {reference_value}')
print(f'Calculated value: {calculated_value}')
print(f'Relative error: {relative_error:.2e}')
print(f'Accuracy target: {metadata["accuracyTarget"]}')

# Verify accuracy
assert relative_error < metadata['accuracyTarget'], f"Error {relative_error:.2e} exceeds target {metadata['accuracyTarget']}"
print('✓ Accuracy target met!')

Reference value: 0.0072973525643
Calculated value: 0.007297340575733252
Relative error: 1.64e-06
Accuracy target: 0.001
✓ Accuracy target met!


In [7]:
# Step 6: Export result
result_data = {
    'id': metadata['id'],
    'symbol': metadata['symbol'],
    'calculated_value': calculated_value,
    'reference_value': reference_value,
    'relative_error': relative_error,
    'unit': metadata['unit'],
    'formula': metadata['formula'],
    'accuracy_met': relative_error < metadata['accuracyTarget']
}

# Save result - try multiple possible paths
output_paths = [
    Path('../constants/results/alpha_result.json'),
    Path('/app/constants/results/alpha_result.json'),
    Path('constants/results/alpha_result.json'),
    Path('./results/alpha_result.json')
]

saved = False
for output_path in output_paths:
    try:
        output_path.parent.mkdir(parents=True, exist_ok=True)
        with open(output_path, 'w') as f:
            json.dump(result_data, f, indent=2)
        print(f'Result saved to {output_path}')
        saved = True
        break
    except Exception as e:
        continue

if not saved:
    print(f'Warning: Could not save result file - tried all paths')

Result saved to /app/constants/results/alpha_result.json
